In [169]:
import os

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

from sklearn.neighbors import KDTree
import numpy as np
from pandas import DataFrame

from sklearn.preprocessing import StandardScaler

import plotly.express as px

import pandas as pd
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate

from sklearn.cluster import KMeans
import numpy as np

import dash_daq as daq

# read enrollment data
path = "demographic-snapshot-2015-16-to-2019-20-(public)_ENInumb.csv"
enrollmentDF = pd.read_csv(path)
enrollmentDF = enrollmentDF[['DBN', 'BN', 'School Name', 'Year', 'Total Enrollment', 'Economic Need Index', 'Enrollment Change']]
enrollmentDF = enrollmentDF[(enrollmentDF['Year'] != '2019-20')] # filter out last year 2020 - we don't have data for it

enrollmentDF['Economic Need Index'] = enrollmentDF['Economic Need Index'].astype(float)
enrollmentDF['Total Enrollment'] = enrollmentDF.apply(lambda row: int(row['Total Enrollment'].replace(',', '')), axis=1)

# read quality report indicator data
file = 'QReports_last3yrs_duplicates_1_transformed.xlsx'
QRreports3yrs_df = pd.read_excel(file, sheet_name='Sheet1')

# groupby BN and take entry from last year
QRreports3yrs_df = QRreports3yrs_df.groupby('BN').tail(1)

# high school achievement
file = '201819hs_sqr_StudentAchievement1BN.xls'
achievementHighDF = pd.read_excel(file, sheet_name='Sheet1')
achievementHighDF = achievementHighDF[["DBN",'BN','School Type','School Name','Student Achievement - Section Score' ]]

# middle school achievement
file = '201819ems_sqr_StudentAchievement1BN.xlsx'
achievementMiddleDF = pd.read_excel(file, sheet_name='Sheet1')
achievementMiddleDF = achievementMiddleDF[["DBN",'BN','School Type','Student Achievement - Section Score' ]]

# union achievements
a = achievementMiddleDF[['DBN', 'BN', 'School Type', 'Student Achievement - Section Score']]
b = achievementHighDF[['DBN', 'BN', 'School Type', 'Student Achievement - Section Score']]
achievementDF = pd.concat([a, b])
achievementDF.drop([0], inplace=True)
achievementDF = achievementDF.rename(columns={'Student Achievement - Section Score': 'Student Achievement'})

print(achievementDF.columns)

# set year
achievementDF['Year'] = "2018-19"

# clean up achievement data
def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def valid(row):
    return isfloat(row['Student Achievement'])

achievementDF = achievementDF[achievementDF.apply(valid, axis=1)]
achievementDF['Student Achievement'] = achievementDF.apply(lambda row: float(row['Student Achievement']), axis=1)
achievementDF

# join enrollment and achievement
joinedDF = enrollmentDF.merge(achievementDF, how='inner', left_on=['DBN', 'Year'], right_on=['DBN', 'Year'])

# read location data
f = "School_location_2019_-_2020.csv"
schoolLocDF = pd.read_csv(f, encoding='latin1')
schoolLocDF = schoolLocDF[['DBN', 'location_type_description', 'LATITUDE', 'LONGITUDE']]
schoolLocDF.drop_duplicates(keep=False, inplace=True)

# create final DF
finalDF = joinedDF.merge(QRreports3yrs_df, how='inner', left_on=['BN_x'], right_on=['BN'])
finalDF = finalDF.merge(schoolLocDF, how='inner', left_on=['DBN'], right_on=['DBN'])

indicator_columns = []

for column in finalDF.columns:
    if column.startswith("Indicator_"):
        indicator_columns.append(column)
        
# finalDF = finalDF[finalDF['Year'] == '2018-19'] #TODO why duplicate years for single school?
finalDF.sort_values(by=['School Name'])

columns = ['School Name', 'location_type_description', 'School Type', 'Student Achievement', 'Total Enrollment', 'Economic Need Index']
columns.extend(indicator_columns)
columns.extend(['LATITUDE', 'LONGITUDE'])
finalDF = finalDF[columns]

# pull out final DF columns
finalDFColumns = finalDF.columns.values.tolist()

# setup features we will use for modeling
features = ["Economic Need Index"]
features.extend(indicator_columns)

# setup linear regression model
df = finalDF
X = df[features] 
y = df['Student Achievement']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

model = linear_model.LinearRegression()
model.fit(X_train, y_train)

# pull out school names
school_names = []

for i, row in finalDF.iterrows():
    school = row['School Name']
    school_names.append({'label': school, 'value': i})

# setup Nearest Neighbor model
X_nearest_columns = ['Total Enrollment', 'Economic Need Index']
X_nearest = finalDF[X_nearest_columns]

#columns are different scales, this normalized them. Standard Scaler (computes avg and sets score to std deviations from avg)
scaler = StandardScaler()
scaler.fit(X_nearest)
X_nearest = scaler.transform(X_nearest)

# Two implementations of nearest neighbor and this is faster one
kdt = KDTree(X_nearest, leaf_size=40, metric='euclidean')

# setup clustering
X_cluster_columns = X_nearest_columns
X_cluster = X_nearest

num_clusters = 5

km = KMeans(
    n_clusters=num_clusters, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)

y_km = km.fit_predict(X_cluster)

# centers transformed back to the original scale
centers = km.cluster_centers_
centers = scaler.inverse_transform(centers)

finalDF['Cluster'] = y_km # add cluster column to the output

# give our a cluster a pretty human-understandable name
def get_cluster_name(cluster_id, centers):
    enrollment = centers[cluster_id][0]
    econ_need = centers[cluster_id][1]
    
    if enrollment >= 3000:
        enrollment_str = "Large size"
    elif enrollment >= 1000:
        enrollment_str = "Medium size"
    else:
        enrollment_str = "Small size"

    if econ_need >= 0.8:
        income_str = "Low income"
    elif econ_need >= 0.6:
        income_str = "Medium income"
    else:
        income_str = "High income"
        
    return f"{cluster_id} ({enrollment_str}, {income_str})"

finalDF['Cluster Description'] = finalDF.apply(lambda row: get_cluster_name(row['Cluster'], centers), axis=1) # change from number to string

# DF for comparable schools table that we display
table_display_columns = finalDF.drop(columns=['LATITUDE', 'LONGITUDE']).columns
table_display_columns

def get_cluster_map(df):
    token = "pk.eyJ1IjoiaW5uYWthcCIsImEiOiJja2ZoZDdwZnYwZDJrMnRtcTE0bGNpN20xIn0.TOk7NlwJcIAlva6Lg5c_hA"
    
    px.set_mapbox_access_token(token)
    category_orders = {'Cluster Description': [get_cluster_name(i, centers) for i in range(num_clusters)]}
    fig = px.scatter_mapbox(df, width=800, height=1000, center={'lat': 40.767245, 'lon': -73.977528}, lat="LATITUDE", lon="LONGITUDE", color='Cluster Description', category_orders=category_orders, size_max=8, zoom=9, hover_name="School Name", hover_data={'Student Achievement':True, 'Total Enrollment':True, 'Economic Need Index':True, 'LATITUDE':False, 'LONGITUDE':False, 'Cluster Description':False})
    return fig

def get_neighbor_map(df, category_orders={}):
    token = "pk.eyJ1IjoiaW5uYWthcCIsImEiOiJja2ZoZDdwZnYwZDJrMnRtcTE0bGNpN20xIn0.TOk7NlwJcIAlva6Lg5c_hA"
    
    px.set_mapbox_access_token(token)
    fig = px.scatter_mapbox(df, width=800, height=1000, lat="LATITUDE", lon="LONGITUDE", size='MarkSize', color='Color', category_orders=category_orders, size_max=8, zoom=10, hover_name="School Name", hover_data={'Student Achievement':True, 'Total Enrollment':True, 'Economic Need Index':True, 'LATITUDE':False, 'LONGITUDE':False, 'Cluster Description':False, 'MarkSize':False, 'Color':False})
    return fig

# helper function to compute correlations
def get_corr_df(df):    
    correlations = []
    
    for cluster in range(num_clusters):
        mydf = df[df['Cluster'] == cluster]
        row = []
        
        y = mydf['Student Achievement']

        for col in indicator_columns:
            c = mydf[col].corr(y)
            row.append(c)   
        correlations.append(row)

    corrDF = pd.DataFrame(correlations, columns=indicator_columns)
    
    # just the 2 decimal point
    corrDF = corrDF.applymap(lambda x: "{:.2f}".format(x))
    
    # convert index to cluster column
    corrDF.reset_index(inplace=True)
    corrDF = corrDF.rename(columns = {'index':'Cluster'})
    
    return corrDF

correlationsDF = get_corr_df(finalDF)

Index(['DBN', 'BN', 'School Type', 'Student Achievement'], dtype='object')


In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# needed by heroku
server = app.server

def get_indicator_label(indicator):
    return html.Label(indicator)
    
def get_indicator_slider(indicator):
    return daq.Slider(
        id=f'{indicator}_id',
        min=0,
        max=4,
        step=1,
        value=0,
        size=800,
        marks={
            0: '0',
            1: '1',
            2: '2',
            3: '3',
            4: '4'
        })

def get_indicator_table():
    d = {'Indicator_1_1':'Curriculum', 'Indicator_1_2':'Pedagogy', 'Indicator_1_3': 'Leveraging Resources', 'Indicator_1_4': 'Positive Learning Environment', 'Indicator_2_2': 'Assessment', 'Indicator_3_1': 'Goals and Action Plans', 'Indicator_3_4': 'High Expectations', 'Indicator_4_1': 'Teacher Support and Supervision', 'Indicator_4_2': 'Teacher Teams and Leadership Development', 'Indicator_5_1': 'Monitoring and Revising Systems'}

    rows = []
    
    for k, v in d.items():
        name = f"{k}: {v}"
        ind_id = k
        row = html.Tr([html.Td(get_indicator_label(name)), html.Td(get_indicator_slider(ind_id))])
        rows.append(row)
        
    return html.Table(rows)

app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Comparable Schools', children=[
            html.B('School Name'),
            dcc.Dropdown(
                id='school_id',
                options=school_names
            ),
            # Br() = line break
            html.Br(),
            # .Div- placeholder

            dcc.Graph(id="map_id"),

            html.Br(),

            html.B('Comparable Schools'),
            dash_table.DataTable(
                id='comp_schools_id',
                columns=[{"name": i, "id": i} for i in table_display_columns],

                sort_action='native',

                style_data_conditional=[
                    {
                        'if': {'row_index': 0},
                        'backgroundColor': 'rgb(255, 253, 184)'
                    }
                ],
            ),

            html.Br(),
        ]),
        dcc.Tab(label='Performance Predictor', children=[
            html.B('Economic Need Index'),
            html.Div(html.P('Economic Need Index is an estimate of the percentage of students at the school facing economic hardship.')),
            html.Div(id='economic_need_index_id'),
        
            html.B('Quality Review Reports'),
            html.Div(html.P('The Quality Review is a school review done by an experienced educator. Using a rubric, the reviewer evaluates the school on 10 indicators that support student achievement.')),

            html.Br(),
            #render indicator and slide     
            
            get_indicator_table(),
            
            html.Br(),

            html.Table([
                html.Tr([ html.Td(html.Label('Predicted Achievement')), html.Td(html.B(id='initial_achievement_id')) ]),
                html.Tr([ html.Td(html.Label('Adjusted Predicted Achievement')), html.Td(html.B(id='predicted_achievement_id')) ]),
            ])
        ]),
        dcc.Tab(label='School Clusters', children=[
            dcc.Graph(id="cluster_map_id", figure=get_cluster_map(finalDF)),

            html.Br(),

            html.B('School Type'),
            dcc.Checklist(
                id='school_type_id',
                options=[
                    {'label': 'High School', 'value': 'High School'},
                    {'label': 'K-8 School', 'value': 'K-8'},
                    {'label': 'Middle School', 'value': 'Middle'},
                    {'label': 'Elementary School', 'value': 'Elementary'}
                ],
                value=['High School', 'K-8', 'Middle', 'Elementary']
            ),

            html.Br(),

            html.B('Education Type'),
            dcc.Checklist(
                id='education_type_id',
                options=[
                    {'label': 'General Academic', 'value': 'General Academic'},
                    {'label': 'Career Technical', 'value': 'Career Technical'},
                    {'label': 'Special Education', 'value': 'Special Education'},
                    {'label': 'Transfer School', 'value': 'Transfer School'}
                ],
                value=['General Academic', 'Career Technical', 'Special Education', 'Transfer School']
            ),

            html.Br(),

            html.B('Key Drivers of Achievement'),

            dash_table.DataTable(
                id='key_drivers_id',
                columns=[{"name": i, "id": i} for i in correlationsDF.columns],
                data=correlationsDF.to_dict('records'),
            ), 
        ]),
    ])
])


# interactive portion: Input = schoolID. If that changes, all declared outputs are affected
# children - in Div container, text is child of container
@app.callback(
    [
        Output('economic_need_index_id', 'children'),
        Output('Indicator_1_1_id', 'value'),
        Output('Indicator_1_2_id', 'value'),
        Output('Indicator_1_3_id', 'value'),
        Output('Indicator_1_4_id', 'value'),
        Output('Indicator_2_2_id', 'value'),
        Output('Indicator_3_1_id', 'value'),
        Output('Indicator_3_4_id', 'value'),
        Output('Indicator_4_1_id', 'value'),
        Output('Indicator_4_2_id', 'value'),
        Output('Indicator_5_1_id', 'value'),
        Output('initial_achievement_id', 'children'),
        Output('comp_schools_id', 'data'),
        Output('map_id', 'figure')
    ],
    [
        Input('school_id', 'value')
    ])
def on_school_select(school_id): #Method for callback: logic for the update
    if not school_id:
        raise PreventUpdate
#         empty_output = [0] * 12
#         empty_output.append([])
#         empty_output.append(None)
#         return empty_output
    
    row = finalDF.loc[school_id]
    
    output = []
    
    # economic achievement output
    output.append(row['Economic Need Index'])
    
    # indicator sliders output
    for col in indicator_columns:
        output.append(row[col])
    
    # inital achievement prediction
    achievement = model.predict([X.loc[school_id]])[0]
    #achievement = row["Student Achievement"] 
    output.append("{:.2f}".format(achievement))
    
    # nearest neighbor search
    school = np.array([row[X_nearest_columns]])
    school = scaler.transform(school)
    comp_school_ids = kdt.query(school, k=10, return_distance=False)[0]
    comp_schools = []

    for school_id in comp_school_ids:
        x = finalDF.iloc[school_id]
        comp_schools.append(x)
    
    # table output
    compSchoolsDF = DataFrame(comp_schools)
    #convert into dash format for table
    output.append(compSchoolsDF.drop(columns=['LATITUDE', 'LONGITUDE']).to_dict('records'))
    
    # map output
    compSchoolsDF['MarkSize'] = 1
    compSchoolsDF['Color'] = 'Comparable schools'
    compSchoolsDF.iloc[0, compSchoolsDF.columns.get_loc('Color')] = 'My school'
    
    # needed to set My school the right color
    category_orders = {'Color': ['Comparable schools', 'My school']}
    
    output.append(get_neighbor_map(compSchoolsDF, category_orders))
    
    return output

# computer predicted achievement based on sliders
@app.callback(
    Output('predicted_achievement_id', 'children'),
    [
        Input('school_id', 'value'),
        Input('Indicator_1_1_id', 'value'),
        Input('Indicator_1_2_id', 'value'),
        Input('Indicator_1_3_id', 'value'),
        Input('Indicator_1_4_id', 'value'),
        Input('Indicator_2_2_id', 'value'),
        Input('Indicator_3_1_id', 'value'),
        Input('Indicator_3_4_id', 'value'),
        Input('Indicator_4_1_id', 'value'),
        Input('Indicator_4_2_id', 'value'),
        Input('Indicator_5_1_id', 'value')
    ])
def on_indicator_slide(school_id, ind11, ind12, ind13, ind14, ind22, ind31, ind34, ind41, ind42, ind51):
    if not school_id:
        return 0
    

    x = X.loc[school_id]
    # setting it to the slider values
    x['Indicator_1.1'] = ind11
    x['Indicator_1.2'] = ind12
    x['Indicator_1.3'] = ind13
    x['Indicator_1.4'] = ind14
    x['Indicator_2.2'] = ind22
    x['Indicator_3.1'] = ind31
    x['Indicator_3.4'] = ind34
    x['Indicator_4.1'] = ind41
    x['Indicator_4.2'] = ind42
    x['Indicator_5.1'] = ind51
    achievement = model.predict([x])[0] #call predicter function
    return "{:.2f}".format(achievement)

# cluster view update on selected filtering
@app.callback(
    [Output('cluster_map_id', 'figure'),
     Output('key_drivers_id', 'data')],
    [Input('school_type_id', 'value'),
     Input('education_type_id', 'value')])
def update(school_types, education_types):
    df = finalDF
    df = df[df['School Type'].isin(school_types)]
    df = df[df['location_type_description'].isin(education_types)]
    data = get_corr_df(df).to_dict('records')
    
    return [get_cluster_map(df), data]

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

/Users/mkapilev/anaconda3/envs/dash2/lib/python3.6/site-packages/ipykernel_launcher.py:234: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/mkapilev/anaconda3/envs/dash2/lib/python3.6/site-packages/ipykernel_launcher.py:235: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/mkapilev/anaconda3/envs/dash2/lib/python3.6/site-packages/ipykernel_launcher.py:236: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/mkapilev/anaconda3